# Import

In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np

# Function

In [30]:
def replace_after(df, condition_type, critical_val, replace_val):
    '''
    給一個dataframe或series，去判斷所有值是否大於、等於、小於、小於等於、大於等於critical_val，
    是的話就把符合條件的值全部替代成replace_val
    如果給定的dataframe不只一個column的話，只要有一個column符合條件，整個row都會換成replace_val 
    '''
    
    if condition_type not in {'<', '<=', '>', '>=', '=='}:
        return 
    if isinstance(df, pd.Series):
        dd = df.to_frame()
    else:
        dd = df.copy()
    condition_str = 'dd' + condition_type + str(critical_val)
    condition = eval(condition_str)
    init_date = dd[dd[condition].any(axis=1)].index
    if len(init_date) == 0:
        return dd
    else:
        init_date = init_date[0]
    init_idx = dd.index.get_loc(init_date)
    dd.iloc[(init_idx + 1):] = replace_val
    
    return dd



# Input parameter

In [10]:
# 股票代號
target_stocks = ['AMD', 'NVDA', 'TSM', 'INTC']

# 產品起始日期 （開始記錄每日收盤價、ko及ki價差）
start_date = '2024-01-12'

# 起始追蹤ko日期 （從這天開始，如果所有股票都Ko，直接結束)
start_trace_date = '2023-10-24'

# 產品預計結束日期 (沒有ko的話的結束日期)
# end_date = datetime.today().strftime('%Y-%m-%d') # 今日
end_date = '2024-01-17'

# 每日收盤價 （起始日期到結束日期的收盤價)
stock_hist = yf.download(target_stocks, start=start_date, end=end_date)['Close']

stock_hist


[*********************100%%**********************]  4 of 4 completed


,AMD,INTC,NVDA,TSM
Date,,,,
2024-01-12,146.559998,47.119999,547.099976,101.239998
2024-01-16,158.740005,47.060001,563.820007,101.669998


# Main

In [32]:
# 得到與ko及ki的差距比例(%)
ko_diff = stock_hist.sub(stock_hist.iloc[0]).div(stock_hist) * 100
ki_diff = stock_hist.sub((stock_hist.iloc[0] * 0.6)).div(stock_hist) * 100

# 換一下column name，對不同dataframe加上後綴
ko_diff = ko_diff.rename(columns={col: col+'_ko_diff' for col in ko_diff.columns})
ki_diff = ki_diff.rename(columns={col: col+'_ki_diff' for col in ki_diff.columns})

# 當每支股票遇到ko差值大於等於0的情況時（目前價>=起始價) 後面都變nan值 （從起始追蹤日開始判斷)
ts_list = []
for c in ko_diff.columns:
    ts = replace_after(ko_diff[c].loc[start_trace_date:], '>=', 0, np.nan)
    ts_list.append(ts)
ko = pd.concat(ts_list, axis=1)
# 當每一支股票ko_diff都變nan時直接結束
ko.dropna(how='all', inplace=True)

# 把起始追蹤日期之前的資料合併進來
init_idx = ko_diff.index.get_loc(start_trace_date)
ko = pd.concat([ko_diff.iloc[:(init_idx)], ko])

# 當其中一支股票的ki差值小於等於0時，後面的值都變nan（目前價<=（起始價×0.6）) (從產品起始日就開始判斷)
ki = replace_after(ki_diff, '<=', 0, np.nan)

# 將產品開始到產品結束（可能會因all ko而提前結束)的資料合併起來
merge_df = ko.merge(ki, left_index=True, right_index=True).merge(stock_hist, left_index=True, right_index=True)

merge_df

,AMD_ko_diff,INTC_ko_diff,NVDA_ko_diff,TSM_ko_diff,AMD_ki_diff,INTC_ki_diff,NVDA_ki_diff,TSM_ki_diff,AMD,INTC,NVDA,TSM
Date,,,,,,,,,,,,
2023-10-10,0.000000,0.000000,0.000000,0.000000,40.000000,40.000000,40.000000,40.000000,109.010002,36.430000,457.980011,90.610001
2023-10-11,-0.646297,1.220176,2.153567,1.510869,39.612222,40.732105,41.292140,40.906521,108.309998,36.880001,468.059998,92.000000
2023-10-12,-0.202226,1.112920,2.443285,1.958448,39.878665,40.667752,41.465971,41.175069,108.790001,36.840000,469.450012,92.419998
2023-10-13,-3.730142,-1.278841,-0.741300,-0.165821,37.761915,39.232695,39.555220,39.900507,105.089996,35.970001,454.609985,90.459999
2023-10-16,-2.395269,0.355583,0.644322,0.668714,38.562839,40.213350,40.386593,40.401228,106.459999,36.560001,460.950012,91.220001
2023-10-17,-3.680809,-1.026065,-4.233239,0.428571,37.791514,39.384361,37.460057,40.257142,105.139999,36.060001,439.380005,91.000000
2023-10-18,-6.694728,-2.216613,-8.536359,-1.127235,35.983163,38.670032,34.878185,39.323659,102.169998,35.639999,421.959991,89.599998
2023-10-19,-6.455079,-2.130648,-8.781264,2.475517,36.126953,38.721611,34.731241,41.485310,102.400002,35.669998,421.010010,92.910004
2023-10-20,-7.072002,-4.324176,-10.657940,0.766616,35.756799,37.405494,33.605236,40.459970,101.809998,34.919998,413.869995,91.309998
